# 🧠 Tutorial 2: Google ADK - Control your LLMs!

## LiteLLM, Parameters and Structured Output

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/)

### 📋 What will you learn in this tutorial?

1. **🔄 Model Flexibility with LiteLLM**
   - Use Claude, GPT, Llama and other models in ADK
   - Multi-provider configuration

2. **⚙️ Fine-tune LLM Behavior**
   - Key parameters: temperature, top_p, max_tokens
   - Use cases for different configurations

3. **📊 Structured Output with Pydantic**
   - Define data schemas
   - Get predictable JSON responses

4. **🚀 Practical and Advanced Examples**
   - Model comparison
   - Complex information extraction

---

### 🎯 Tutorial Objective

After completing this tutorial, you will be able to:
- Integrate any LLM into your ADK agents
- Fine-tune model behavior
- Get structured and validated responses

**Prerequisites:**
- Having completed ADK Tutorial 1
- API Keys for the models you want to use

## 🔧 Initial Setup

### Installing Dependencies

In [ ]:
# Install Google ADK with LiteLLM support
print("📦 Installing Google ADK with LiteLLM...")
!pip install -q google-adk==1.4.2
!pip install -q litellm==1.73.0
!pip install -qU python-dotenv pydantic

print("\n✅ Installation completed!")

# Check versions
import sys
print(f"\n🐍 Python: {sys.version.split()[0]}")
!pip show google-adk litellm pydantic | grep -E "Name:|Version:"

### API Keys Configuration

For this tutorial, you'll need at least one API Key. You can get them from:
- **Google AI Studio**: [https://aistudio.google.com/apikey](https://aistudio.google.com/apikey)
- **Anthropic (Claude)**: [https://console.anthropic.com/](https://console.anthropic.com/)
- **OpenAI**: [https://platform.openai.com/](https://platform.openai.com/)

#### Option 1: Enter them directly

In [ ]:
import os
from getpass import getpass

print("🔑 API Keys Configuration\n")
print("Enter the API Keys you have available (press Enter to skip):\n")

# Google API Key (required for base examples)
if 'GOOGLE_API_KEY' not in os.environ:
    google_key = getpass("Google API Key (required): ")
    if google_key:
        os.environ['GOOGLE_API_KEY'] = google_key
        os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'

# Anthropic API Key (optional)
if 'ANTHROPIC_API_KEY' not in os.environ:
    anthropic_key = getpass("Anthropic API Key (optional): ")
    if anthropic_key:
        os.environ['ANTHROPIC_API_KEY'] = anthropic_key

# OpenAI API Key (optional)
if 'OPENAI_API_KEY' not in os.environ:
    openai_key = getpass("OpenAI API Key (optional): ")
    if openai_key:
        os.environ['OPENAI_API_KEY'] = openai_key

# Check configuration
print("\n📋 API Keys Status:")
print(f"   Google: {'✅' if os.environ.get('GOOGLE_API_KEY') else '❌'}")
print(f"   Anthropic: {'✅' if os.environ.get('ANTHROPIC_API_KEY') else '❌'}")
print(f"   OpenAI: {'✅' if os.environ.get('OPENAI_API_KEY') else '❌'}")

#### Option 2: Use dotenv

In [ ]:
from dotenv import load_dotenv
# Load environment variables from .env if it exists
load_dotenv(override=True)

## 🔄 Part 1: Model Flexibility with LiteLLM

### What is LiteLLM?

LiteLLM is a library that provides a unified interface for 100+ different language models. It acts as a "universal translator" between ADK and various LLM providers.

### Advantages of using LiteLLM:

- 🧪 **Easy experimentation**: Test different models without changing your code
- 💰 **Cost optimization**: Choose the most economical model for each task
- 🔓 **No vendor lock-in**: Freedom to change providers
- 🎯 **Specialized models**: Use the best model for each case

### Create Agents with Different Models

In [ ]:
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.agents.llm_agent import LlmAgent
from google.genai import types


async def call_agent_async(query: str, runner, user_id, session_id):
    """Sends a query to the agent and prints the final response."""
    print(f"\n>>> User query: {query}")

    # Prepare the user message in ADK format
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "The agent did not produce a final response." # Default value

    # Key concept: run_async executes the agent's logic and generates events.
    # We iterate through events to find the final response.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # You can uncomment the line below to see *all* events during execution
        # print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

        # Key concept: is_final_response() marks the message that concludes the turn.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Assume the text response is in the first part
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Handle possible errors/escalations
                final_response_text = f"The agent escalated: {event.error_message or 'No specific message.'}"
            # Add more validations here if needed (e.g., specific error codes)
            break # Stop processing events once final response is found

    print(f"<<< Agent response: {final_response_text}")

### First Google's Gemini

In [ ]:
MODEL_GEMINI = "gemini-2.5-flash"

# Example: Defining the basic Agent
proverbs_agent = LlmAgent(
    model=MODEL_GEMINI,
    name="proverbs_agent",
    description="completes the proverbs that the user starts"
)

In [ ]:
session_service = InMemorySessionService()

APP_NAME = "test_gemini"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
# Runner: This is the main component that manages the interaction with the agent.
runner_gemini = Runner(agent=proverbs_agent,app_name=APP_NAME,session_service=session_service)

In [ ]:
await call_agent_async("Don't look a gift horse...",
                           runner=runner_gemini,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

## Let's go with OpenAI

In [ ]:
from google.adk.models.lite_llm import LiteLlm
openai_model = LiteLlm("openai/gpt-4o")

In [ ]:
proverbs_agent_openai = LlmAgent(
    model=openai_model,
    name="proverbs_agent",
    description="completes the proverbs that the user starts",
)

APP_NAME = "test_openai"
USER_ID = "user_2"
SESSION_ID = "session_002" # Using a fixed ID for simplicity

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_openai = Runner(agent=proverbs_agent_openai,app_name=APP_NAME,session_service=session_service)

In [ ]:
await call_agent_async("The early bird...",
                           runner=runner_openai,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

## Let's try Anthropic

In [ ]:
anthropic_model = LiteLlm("anthropic/claude-3-5-sonnet-20250116")
proverbs_agent_claude = LlmAgent(
    model=anthropic_model,
    name="proverbs_agent",
    description="completes the proverbs that the user starts",
)

APP_NAME = "test_claude"
USER_ID = "user_3"
SESSION_ID = "session_003" # Using a fixed ID for simplicity

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_claude = Runner(agent=proverbs_agent_claude,app_name=APP_NAME,session_service=session_service)

In [ ]:
await call_agent_async("When in Rome...",
                           runner=runner_claude,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

## Even with Azure OpenAI

In [ ]:
azure_model = LiteLlm("azure/gpt-4o")
proverbs_agent_azure = LlmAgent(
    model=azure_model,
    name="proverbs_agent",
    description="completes the proverbs that the user starts",
)

APP_NAME = "test_azure"
USER_ID = "user_4"
SESSION_ID = "session_004" # Using a fixed ID for simplicity


session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_azure = Runner(agent=proverbs_agent_azure,app_name=APP_NAME,session_service=session_service)

In [ ]:
await call_agent_async("A new broom...",
                           runner=runner_azure,
                           user_id=USER_ID,
                           session_id=SESSION_ID)

## Even local models

In [ ]:
ollama_model = LiteLlm("ollama/gemma2:2b")
proverbs_agent_ollama = LlmAgent(
    model=ollama_model,
    name="proverbs_agent",
    description="completes the proverbs that the user starts",
)

APP_NAME = "test_ollama"
USER_ID = "user_5"
SESSION_ID = "session_005" # Using a fixed ID for simplicity


session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_ollama = Runner(agent=proverbs_agent_ollama,app_name=APP_NAME,session_service=session_service)

In [ ]:
await call_agent_async("A bird in the hand...",
                        runner=runner_ollama,
                        user_id=USER_ID,
                        session_id=SESSION_ID)

## ⚙️ Part 2: Fine-tuning Behavior with Parameters

### Key LLM Parameters

Parameters allow you to control how models generate text:

1. **🌡️ Temperature (0.0 - 2.0)**
   - Low (0.0-0.3): Deterministic and conservative responses
   - Medium (0.4-0.7): Balance between consistency and creativity
   - High (0.8-2.0): Creative and diverse responses

2. **🎯 Top-p (0.0 - 1.0)**
   - Controls "nucleus sampling"
   - 0.9 = considers tokens that sum to 90% probability
   - Alternative to temperature (use one or the other)

3. **📏 Max Output Tokens**
   - Limits response length
   - Useful for controlling costs and conciseness

In [ ]:
# Creative Agent (high temperature)
creative_agent = LlmAgent(
    name="CreativeAgent",
    model=azure_model,
    description="Agent configured for maximum creativity",
    generate_content_config=types.GenerateContentConfig(
        temperature= 1.5,          # High creativity
        max_output_tokens = 1000,    # Moderate responses
        top_k= 40                  # Wide vocabulary
     ),
    instruction=(
        "You are a creative and imaginative writer."
        "Generate original and surprising ideas."
        "Use metaphors, analogies and colorful language."
    )
)

# Technical Agent (low temperature)
technical_agent = LlmAgent(
    name="TechnicalAgent",
    model=azure_model,
    description="Agent configured for technical precision",
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.1,          # Very deterministic
        max_output_tokens= 150    # Concise responses
    ),
    instruction=(
        "You are a precise and factual technical expert."
        "Provide accurate and verifiable information."
        "Avoid speculation and stick to facts."
    )
)

# Balanced Agent (medium configuration)
balanced_agent = LlmAgent(
    name="BalancedAgent",
    model=azure_model,
    description="Agent with balanced configuration",
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.7,          # Balance
        max_output_tokens= 300    # Flexible length
    ),
    instruction=(
        "You are a versatile and adaptable assistant."
        "Provide useful and well-structured responses."
        "Adapt your style according to context."
    )
)

# Ultra-Concise Agent (limited tokens)
concise_agent = LlmAgent(
    name="ConciseAgent",
    model=azure_model,
    description="Agent for ultra-brief responses",
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.3,
        max_output_tokens= 50     # Very limited
    ),
    instruction=(
        "Respond extremely concisely."
        "Maximum 2-3 sentences per response."
        "Get straight to the point, no detours."
    )
)

print("🎛️ Agents with different parameters created:")
print(" • CreativeAgent (temp=1.5)")
print(" • TechnicalAgent (temp=0.1)")
print(" • BalancedAgent (temp=0.7)")
print(" • ConciseAgent (max_tokens=50)")

### 🧪 Demonstration: Effect of Parameters

In [ ]:
APP_NAME = "test_creative_agent"
USER_ID = "user_6"
SESSION_ID = "session_006" # Using a fixed ID for simplicity

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_creative = Runner(agent=creative_agent,app_name=APP_NAME,session_service=session_service)

await call_agent_async("Write a poem about the full moon",
                        runner=runner_creative,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


In [ ]:
APP_NAME = "test_concise_agent"
USER_ID = "user_6"
SESSION_ID = "session_006" # Using a fixed ID for simplicity


session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_concise = Runner(agent=concise_agent,app_name=APP_NAME,session_service=session_service)

await call_agent_async("Write a poem about the full moon",
                        runner=runner_concise,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


### 📊 Parameter Usage Guide

Recommendations by use case:

In [ ]:
# Create visual parameter guide
import pandas as pd

# Create recommendations table
parameter_guide = pd.DataFrame([
    {"Use Case": "Technical Documentation", 
     "Temperature": "0.1-0.3", 
     "Max Tokens": "500-1000", 
     "Top-p": "0.9-0.95",
     "Reason": "Precision and consistency"},
    
    {"Use Case": "Creative Writing", 
     "Temperature": "0.8-1.5", 
     "Max Tokens": "200-500", 
     "Top-p": "0.95-1.0",
     "Reason": "Originality and variety"},
    
    {"Use Case": "Customer Service Chatbot", 
     "Temperature": "0.5-0.7", 
     "Max Tokens": "100-200", 
     "Top-p": "0.9",
     "Reason": "Balance and efficiency"},
    
    {"Use Case": "Data Analysis", 
     "Temperature": "0.1-0.2", 
     "Max Tokens": "300-500", 
     "Top-p": "0.95",
     "Reason": "Accuracy in results"},
    
    {"Use Case": "Brainstorming", 
     "Temperature": "1.0-1.8", 
     "Max Tokens": "150-300", 
     "Top-p": "0.95-1.0",
     "Reason": "Maximum idea diversity"}
])

print("📊 PARAMETER GUIDE BY USE CASE\n")
print(parameter_guide.to_string(index=False))

print("\n\n⚠️ Important notes:")
print("   • These are suggested ranges, experiment according to your needs")
print("   • Don't use temperature and top-p simultaneously at extreme values")
print("   • Cost increases with max_tokens, use prudently")
print("   • Some models have specific limits, check documentation")

## 📊 Part 3: Structured Output with Pydantic

### Why Structured Output?

Structured output is crucial when you need to:
- 🔧 Integrate AI responses with other systems
- 💾 Store data in databases
- 🎯 Guarantee consistent format
- ✅ Validate extracted information

### Pydantic: The Solution

Pydantic allows defining data schemas with:
- Python type hints
- Automatic validation
- JSON serialization
- Clear documentation for the LLM

In [ ]:
# Import Pydantic and create data models
from pydantic import BaseModel, Field
from typing import List, Optional

print("📚 Creating Pydantic models for structured output...\n")

# Model 1: Product Information
class ProductInformation(BaseModel):
    """Schema for extracting product information"""
    name: str = Field(description="Complete product name")
    brand: Optional[str] = Field(None, description="Product brand")
    price: Optional[float] = Field(None, description="Price in USD")
    features: List[str] = Field(
        default_factory=list,
        description="List of main features"
    )
    available: bool = Field(True, description="If it's available")
    category: Optional[str] = Field(None, description="Product category")

# Model 2: Sentiment Analysis
class SentimentAnalysis(BaseModel):
    """Schema for text sentiment analysis"""
    sentiment: str = Field(
        description="General sentiment: positive, negative or neutral"
    )
    confidence: float = Field(
        description="Analysis confidence level (0.0 to 1.0)",
        ge=0.0, le=1.0
    )
    emotions: List[str] = Field(
        default_factory=list,
        description="Emotions detected in the text"
    )
    positive_aspects: List[str] = Field(
        default_factory=list,
        description="Positive aspects mentioned"
    )
    negative_aspects: List[str] = Field(
        default_factory=list,
        description="Negative aspects mentioned"
    )

# Model 3: Event Extraction
class Event(BaseModel):
    """Information about an event"""
    title: str = Field(description="Event title")
    date: Optional[str] = Field(None, description="Event date (YYYY-MM-DD)")
    time: Optional[str] = Field(None, description="Event time (HH:MM)")
    location: Optional[str] = Field(None, description="Event location")
    participants: List[str] = Field(
        default_factory=list,
        description="List of participants"
    )
    description: Optional[str] = Field(None, description="Event description")

class EventList(BaseModel):
    """List of extracted events"""
    events: List[Event] = Field(
        default_factory=list,
        description="List of all events found"
    )
    total_events: int = Field(
        description="Total number of events found"
    )

print("✅ Pydantic models created:")
print("   1. ProductInformation - For extracting product data")
print("   2. SentimentAnalysis - For opinion analysis")
print("   3. Event/EventList - For extracting event information")

# Usage example
print("\n📋 Schema example (ProductInformation):")
print(ProductInformation.model_json_schema()['properties'])

## Creating agents with structured output

In [ ]:
# Create agents with structured output

# Product Extractor Agent
product_extractor_agent = LlmAgent(
    name="ProductExtractor",
    model="gemini-2.5-flash",  # Pro handles structured output better
    description="Extracts structured product information",
    output_schema=ProductInformation,  # Structured output!
    output_key='ProductInformation',  # Output key
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.1,
        max_output_tokens= 300
    ),
    instruction=(
        "Extract product information from the provided text."
        "Follow the defined schema exactly."
        "If you don't find some data, use None or empty list as appropriate."
        "Be precise with prices and features."
    )
)

# Sentiment Analyzer Agent
sentiment_agent = LlmAgent(
    name="SentimentAnalyzer",
    model=openai_model,
    description="Analyzes sentiment and emotions in texts",
    output_schema=SentimentAnalysis,  # Structured output!
    output_key='SentimentAnalysis',  # Output key
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.3,
        max_output_tokens= 300
    ),
    instruction=(
        "Analyze the sentiment of the provided text."
        "Identify specific emotions present."
        "List positive and negative aspects mentioned."
        "Assign a confidence level to your analysis (0.0 to 1.0)."
    )
)

# Event Extractor Agent
event_agent = LlmAgent(
    name="EventExtractor",
    model=anthropic_model,
    description="Extracts event information from text",
    output_schema=EventList,  # Structured output with list!
    output_key='EventList',  # Output key
    generate_content_config=types.GenerateContentConfig(
        temperature= 0.2,
        max_output_tokens= 500
    ),
    instruction=(
        "Extract ALL events mentioned in the text."
        "For each event, capture all available information."
        "Dates should be in YYYY-MM-DD format."
        "Times in HH:MM format."
        "If there are multiple events, include them all in the list."
    )
)

print("🎯 Agents with structured output created:")
print("   • ProductExtractor → ProductInformation")
print("   • SentimentAnalyzer → SentimentAnalysis")
print("   • EventExtractor → EventList")

### Testing the first product agent

In [ ]:
APP_NAME = "output_1"
USER_ID = "user_7"
SESSION_ID = "session_007" # Using a fixed ID for simplicity

product_text = """
    The new iPhone 15 Pro Max from Apple is now available. 
    Priced at $1,199, it includes a 48MP camera, 6.7-inch display,
    A17 Pro chip and long-lasting battery. Available in titanium.
    """

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_products = Runner(agent=product_extractor_agent,app_name=APP_NAME,session_service=session_service)

await call_agent_async(product_text,
                        runner=runner_products,
                        user_id=USER_ID,
                        session_id=SESSION_ID)

### Testing the second sentiment agent

In [ ]:
APP_NAME = "output_2"
USER_ID = "user_7"
SESSION_ID = "session_007" # Using a fixed ID for simplicity

opinion_text = """
    I love my new laptop! The speed is incredible and the screen is beautiful.
    However, the battery doesn't last as long as I expected and the price was somewhat high.
    Overall, I'm satisfied with the purchase.
    """

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_sentiment = Runner(agent=sentiment_agent,app_name=APP_NAME,session_service=session_service)

await call_agent_async(opinion_text,
                        runner=runner_sentiment,
                        user_id=USER_ID,
                        session_id=SESSION_ID)

### Testing the third event agent

In [ ]:
APP_NAME = "output_3"
USER_ID = "user_7"
SESSION_ID = "session_007" # Using a fixed ID for simplicity

events_text = """
    Reminder: The AI conference will be on March 15, 2024 at 10:00 AM
    at the Convention Center. Dr. Smith and Dr. Johnson will participate.
    Afterwards, there will be a practical workshop at 2:00 PM at the same location.
    """

session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
runner_events = Runner(agent=event_agent,app_name=APP_NAME,session_service=session_service)

await call_agent_async(events_text,
                        runner=runner_events,
                        user_id=USER_ID,
                        session_id=SESSION_ID)

____

## 🎓 TUTORIAL 2 SUMMARY: LLM CONTROL
---

### 1️⃣ LiteLLM - Model Flexibility
- ✅ Use any model with the `litellm/` prefix
- ✅ Support for **100+ different models**
- ✅ Easy switching between providers
- ✅ **Cost and performance** optimization

---

### 2️⃣ LLM Parameters - Control
- ✅ `temperature`: controls **creativity vs consistency**
- ✅ `max_tokens`: limits response **length** and helps control costs
- ✅ `top_p`: alternative to `temperature` for sampling
- ✅ Configuration tailored to **use case**

---

### 3️⃣ Structured Output - Predictable Responses
- ✅ Use **Pydantic** to define response schemas
- ✅ **Automatic validation** of model-generated data
- ✅ Easy integration with other systems
- ✅ Production of **consistent and typed JSON**

---

---

## 🎉 Congratulations!

You have completed Google ADK Tutorial 2. Now you have the power to:
- Integrate any LLM into your agents
- Fine-tune their behavior
- Get structured and validated responses

**Next step**: Tutorial 3 - Tools  🛠️

---

**Questions?** Leave them in the video comments or check the official documentation.

**Happy coding with ADK!** 🚀

## 📚 Additional Resources

### Useful Links

- **ADK Documentation**: [https://google.github.io/adk-docs/](https://google.github.io/adk-docs/)
- **LiteLLM Docs**: [https://docs.litellm.ai/](https://docs.litellm.ai/)
- **Pydantic Docs**: [https://docs.pydantic.dev/](https://docs.pydantic.dev/)
- **Supported Models**: [Complete LiteLLM List](https://docs.litellm.ai/docs/providers)